# Pydeck inntekt kommuner

> https://data.ssb.no/api/v0/dataset/56957.csv?lang=no

In [1]:
import json
import os
import requests
import deetly
import geopandas as gp
import numpy as np
import pandas as pd
import pydeck

In [2]:
MAPBOX_TOKEN = os.environ['MAPBOX_TOKEN'] 

In [3]:
MAP_URL = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/counties_2018_s_geojson.json'

In [4]:
package = {
    "name":"Norwegian Counties Income Above 750k", 
    "keywords": ["examples", "pydeck", "counties", "maps"]
}

ds = deetly.story(package)

In [5]:
df = pd.read_csv('https://data.ssb.no/api/v0/dataset/56957.csv?lang=no', encoding='latin1', sep=';')

In [6]:
df['kommunenummer']=df['region'].apply(lambda x: x[0:4])
df.rename(columns={'07182: Husholdninger, etter region, statistikkvariabel og år': 'prosent'}, inplace=True)

In [7]:
df.head(1)

,region,statistikkvariabel,år,prosent,kommunenummer
0,0101 Halden (-2019),Antall husholdninger,2006,12575,0101


In [8]:
df['statistikkvariabel'].unique()

array(['Antall husholdninger',
       'Inntekt etter skatt under 150 000 kr, prosent',
       'Inntekt etter skatt 150 000 - 249 999 kr, prosent',
       'Inntekt etter skatt 250 000 - 349 999 kr, prosent',
       'Inntekt etter skatt 350 000 - 449 999 kr, prosent',
       'Inntekt etter skatt 450 000 - 549 999 kr, prosent',
       'Inntekt etter skatt 550 000 - 749 999 kr, prosent',
       'Inntekt etter skatt 750 000 kr og over, prosent'], dtype=object)

In [9]:
df['region'].unique()

array(['0101 Halden (-2019)', '0104 Moss (-2019)',
       '0105 Sarpsborg (1992-2019)', '0106 Fredrikstad (1994-2019)',
       '0111 Hvaler (-2019)', '0118 Aremark (-2019)',
       '0119 Marker (1964-2019)', '0121 Rømskog (-2019)',
       '0122 Trøgstad (-2019)', '0123 Spydeberg (-2019)',
       '0124 Askim (-2019)', '0125 Eidsberg (-2019)',
       '0127 Skiptvet (-2019)', '0128 Rakkestad (-2019)',
       '0135 Råde (-2019)', '0136 Rygge (-2019)',
       '0137 Våler (Østfold) (-2019)', '0138 Hobøl (-2019)',
       '0211 Vestby (-2019)', '0213 Ski (1931-2019)', '0214 Ås (-2019)',
       '0215 Frogn (-2019)', '0216 Nesodden (-2019)',
       '0217 Oppegård (1915-2019)', '0219 Bærum (-2019)',
       '0220 Asker (-2019)', '0221 Aurskog-Høland (1966-2019)',
       '0226 Sørum (-2019)', '0227 Fet (-2019)',
       '0228 Rælingen (1929-2019)', '0229 Enebakk (-2019)',
       '0230 Lørenskog (1908-2019)', '0231 Skedsmo (-2019)',
       '0233 Nittedal (-2019)', '0234 Gjerdrum (-2019)',
       '023

In [10]:
df_high = df[(df['statistikkvariabel']=='Inntekt etter skatt 750 000 kr og over, prosent') & (df['år']==2019)]

In [11]:
df_high.head(1)

,region,statistikkvariabel,år,prosent,kommunenummer
111,0101 Halden (-2019),"Inntekt etter skatt 750 000 kr og over, prosent",2019,25,0101


In [12]:
df_geo = gp.read_file(MAP_URL)
df_high_geo = df_geo.merge(df_high, on='kommunenummer')
df_high_geo.head(1)

,id,kommunenummer,oppdateringsdato,navn,area_kommune,fylke_nummer_2018,fylke_nummer_2019,fylke_navn_2019,fylke_nummer_2020,fylke_navn_2020,geometry,region,statistikkvariabel,år,prosent
0,0,0101,2018-12-10T00:00:00,Halden,0.100015,01,01,ØSTFOLD,30,VIKEN,"POLYGON ((11.55794 59.25961, 11.71052 59.03368...",0101 Halden (-2019),"Inntekt etter skatt 750 000 kr og over, prosent",2019,25


In [15]:
# Map centered on Trondheim, Norway
view_state_norway = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 45,
    bearing= 0)

In [16]:
df_geo = gp.read_file(MAP_URL)

layer = pydeck.Layer(
    'GeoJsonLayer',
    df_high_geo,
    stroked = False,
    filled = True,
    extruded = True,
    wireframe = False,
    elevationScale = 8000,
    elevation_range=[20, 40],
    get_elevation = 'prosent',
    width_min_pixels=5,
    get_fill_color='[255, 255, 2.5*prosent]',
    getLineColor = '[255, 255, 2.5*prosent]',
    pickable=True
)


tooltip={
        'html': '<b>Kommune: </b>{navn}</br>\
                Fylke: {fylke_navn_2020}</br>\
                Andel: {prosent}',
        'style': {
            'color': 'black',
            'backgroundColor':'#efefef',
            'zIndex': 999
        }
    }

# Render
fig = pydeck.Deck(layers=[layer],
                  initial_view_state=view_state_norway,
                  #api_keys={"mapbox":MAPBOX_TOKEN},
                  tooltip=tooltip
)

fig.show()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"bea…

In [18]:
# Add figure to data package
ds.pydeck(fig, "Andel med inntekt over 750k",showMap=False, tooltip=json.dumps(tooltip))

## Publish

In [19]:
# publish datastory
ds.publish()

Publishing story Norwegian Counties Income Above 750k to the examples space...
Published at: https://public.deetly.com/examples/ddb4128518ee150f3172271d3488521b 

Metadata: https://storage.googleapis.com/deetly/examples/ddb4128518ee150f3172271d3488521b/datapackage.json 

